In [1]:
import os
import pytesseract
from pdf2image import convert_from_path
import string 
from nltk.stem.snowball import SnowballStemmer
from googletrans import Translator
import pandas as pd
import string 
from nltk.tokenize import sent_tokenize
from PyDictionary import PyDictionary
import json

code_list1 = list()
word_list1 = list()
def vocab_cam(file_name1,trans_lang):
    def File_Content(file_name):
        file_str = ''

        if file_name.endswith('.pdf'):
            pdf_file = convert_from_path(file_name)

            for i in range(len(pdf_file)):
                pdf_file[i].save(f'page{str(i + 1)}.jpg', 'JPEG')   # Saves pages as images in the pdf
                file = f'page{str(i + 1)}.jpg'
                file_str += pytesseract.image_to_string(file)
                # os.remove(file)   # Deletes saved image files
        elif file_name.endswith(('.jpg', '.png')):
            file_str = pytesseract.image_to_string(file_name)   
        return file_str
    file_cont= File_Content(file_name1)
    
    with open("Deneme.txt","a",encoding="utf-8") as file:
        file.write(file_cont)  
    with open("Deneme.txt","r+",encoding="utf-8") as file:
        content= file.readlines()
        parag=  file.read()
        untitled=""
        for i in content:
            if(i.isupper()==True or (i.istitle()==True)):
                untitled += " "
            else:
                untitled += i + " "
    with open("Computer Reading Son.txt","a",encoding="utf-8") as file1:
        for i in untitled:
            file1.write(i)
    with open("Computer Reading Son.txt","r",encoding="utf-8") as file:
        contents=file.read()
        content_clear =contents.replace("\n","")
        content_clear = content_clear.replace("/n","")
        def punctuation_clear(text):
            result = ""
            for i in text:
                if( i not in string.punctuation and i.isalpha() != False or i == " "  ):
                    result+= i 
                else:
                    result+= ""
            return result                
        
        def punctuation_clear_list(text):
            result = ""
            text=text.lower()
            for i in text:
                for a in i:
                    if( a not in string.punctuation and a.isalpha() or a == "." or a == " " ):
                        result+= a 
            return result
        content_clears = punctuation_clear(content_clear)
        content_split_sentence = punctuation_clear_list(content_clear).split(".")
        content_word_split= set(content_clears.split())
        word_list= list()
        
        for i in content_word_split:
            if(len(i)>=3):
                i = i.lower()
                i=punctuation_clear(i)
                word_list.append(i)
        say1=0
        for q in word_list:
            say1+=1
            word_list1.append(q)
        print("Kelime Listesi : ",say1)
    
    p=0
    sentence_list_last = list()
    for w in word_list:
        
        num=0
        for r in content_split_sentence:
            sentence = r.split()
            for e in sentence:
                if w==e:
                    sentence_list_last.append(r)
                    num+=1
                if num==1:
                    break
            if num==1:
                break
    say2=0
    for q in sentence_list_last:
        say2+=1
    print("Cümle Listesi : ",say2)            
    content_clear_split_last = list()
    from nltk.stem.snowball import SnowballStemmer
    stem_list=[]

    for i in word_list:
        stemmer = SnowballStemmer("english")
        stem_list.append(stemmer.stem(i))
    
    say3=0
    for q in stem_list:
        say3+=1
    print("kök Listesi : ",say3)
    from googletrans import Translator
    translator = Translator()
    ceviri_list= list()
    for i in word_list:
        translation = translator.translate(i,dest = trans_lang)
        ceviri_list.append(translation.text)
    say4=0
    for q in ceviri_list:
        say4+=1
    print("çeviri Listesi : ",say4)
    from nltk.corpus import state_union
    import nltk
    from nltk.tokenize import PunktSentenceTokenizer
    code_list = list()
    for i in word_list:
        words = nltk.word_tokenize(i)
        tagged = nltk.pos_tag(words)
        code_list.append(tagged[0][1])
    say5=0
    for q in code_list:
        say5+=1
        code_list1.append(q)
    print("kod Listesi : ",say5)
    dictionary = PyDictionary()
    pydictlist = list()
    counter = 0
    for i in code_list:
        try:
            if(i=="NN" or i=="NNS" or i=="NNP" or i == "NNPS"):
                definition=dictionary.meaning(word_list[counter],disable_errors=True)["Noun"][0]
                pydictlist.append(definition)
            elif(i == "VB" or i == "VBD" or i == "VBG" or i == "VBN" or i == "VBP" or i  == "VBZ" ):
                definition=dictionary.meaning(word_list[counter],disable_errors=True)["Verb"][0]
                pydictlist.append(definition)
            else:
                pydictlist.append("None")
        except TypeError:
            pydictlist.append("None")
        except KeyError:
            pydictlist.append("None")
        except ValueError:
            pydictlist.append("None")
        except IndexError:
            pydictlist.append("None")
        counter +=1
    
    dict = {"Code:":code_list,"Raw":word_list,"Root":stem_list,"Translation":ceviri_list,"Eng Translation":pydictlist,"Sentence":sentence_list_last,}
    df= pd.DataFrame(dict)
    df.to_json("Computer Reading Translate.json")
    return df
        




In [2]:
vocab_cam('Computer Reading.pdf',"tr")

Kelime Listesi :  410
Cümle Listesi :  410
kök Listesi :  410
çeviri Listesi :  410
kod Listesi :  410


,Code:,Raw,Root,Translation,Eng Translation,Sentence
0,IN,from,from,itibaren,None,some operating systems have graphical user in...
1,NNS,looks,look,görünüyor,the feelings expressed on a person's face,the main device for inputting the data is a t...
2,NN,variety,varieti,Çeşitlilik,a collection containing a variety of sorts of ...,they can be used in a very wide variety of si...
3,IN,between,between,arasında,None,an internet service called ftp file transfer ...
4,NNS,allows,allow,izin verir,None,a special set of programs called an operating...
...,...,...,...,...,...,...
405,NN,equipment,equip,teçhizat,an instrumentality needed for an undertaking o...,the equipment is known as the hardware and t...
406,NN,found,found,bulundu,food and lodging provided in addition to money,they can be used in a very wide variety of si...
407,VBG,videoconferencing,videoconferenc,video konferans,None,video ﬁles attached to simple email text mess...
408,NN,printing,print,baskı,text handwritten in the style of printed matter,one of the most common peripherals is a print...
